In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [9]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [11]:
# drop the irrelevant column
data =  data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.shape

(10000, 11)

In [12]:
## Encode categorical variables
# label_encoder_gender=LabelEncoder()
# data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
# data
data["Gender"] = (
    data["Gender"]
    .str.strip()
    .str.capitalize()
    .map({"Female": 0, "Male": 1})
)


# Method 2
# data['gender'] = data['gender'].map({'Male': 1, 'Female': 0})


In [14]:
data.Gender

0       0
1       0
2       0
3       0
4       0
       ..
9995    1
9996    1
9997    0
9998    1
9999    0
Name: Gender, Length: 10000, dtype: int64

In [81]:
# data[["Geography"]]

In [13]:
# data["Geography"]

In [15]:
# encode Geography
onehot_encoder_geography = OneHotEncoder()
encoded_geo = onehot_encoder_geography.fit_transform(data[["Geography"]]).toarray()
encoded_geo

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [16]:
onehot_encoder_geography.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [17]:
encoded_geo_df = pd.DataFrame(encoded_geo, columns=onehot_encoder_geography.get_feature_names_out(["Geography"]))
encoded_geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [18]:
data = data.drop(["Geography"], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,42,2,0.00,1,1,1,101348.88,1
1,608,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,39,1,0.00,2,0,0,93826.63,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0


In [19]:
# concating with the data and removing the unencoded one
data = pd.concat([data, encoded_geo_df], axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [20]:
# Feature division and standardization
X = data.drop(["Exited"], axis=1)
y = data["Exited"]



In [21]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
# Standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
# Saving the pickle files
import os
import pickle

os.makedirs("pkl_files", exist_ok=True)

# # 1. labelEncoder
# with open('pkl_files/label_encoder_gender.pkl','wb') as file:
#     pickle.dump(label_encoder_gender,file)
    
# 2. OneHotEncoder
with open("pkl_files/onehot_encoder_geography.pkl", "wb") as file:
    pickle.dump(onehot_encoder_geography, file) 
    
# 3. Sacler
with open("pkl_files/scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)    

In [24]:
# ANN implementation
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import datetime


In [25]:
(X_train.shape[1], )

(12,)

In [26]:
model = Sequential([
    Dense(64, activation="relu",  input_shape = (X_train.shape[1],)),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# Options for optimizer and losses
opt = tensorflow.optimizers.Adam(learning_rate=0.001)
los= tensorflow.losses.BinaryCrossentropy()
los, opt

(<keras.src.losses.BinaryCrossentropy at 0x21139027390>,
 <keras.src.optimizers.adam.Adam at 0x2113907fed0>)

In [28]:
# Compile the model to forward and backword propogation followup
model.compile(optimizer=opt, loss=los, metrics=["accuracy"])

In [29]:
# setup tensorflow_callback
import datetime
import tensorflow as tf

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

tensorflow_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1
)


In [30]:
# setup the early Stopping
early_stopping_callback = EarlyStopping(monitor="val_loss", restore_best_weights=True) #validation_loss

In [31]:
# TRain the model
history = model.fit(X_train, y_train, 
                    validation_data = (X_test, y_test),
                    epochs = 100,
                    callbacks = [tensorflow_callback, early_stopping_callback]
                    )
model.save("model.h5")


Epoch 1/100


250/250 [==============================] - 1s 2ms/step - loss: 0.4580 - accuracy: 0.8029 - val_loss: 0.4003 - val_accuracy: 0.8270
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3929 - accuracy: 0.8359 - val_loss: 0.3625 - val_accuracy: 0.8495
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3580 - accuracy: 0.8546 - val_loss: 0.3533 - val_accuracy: 0.8550
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3460 - accuracy: 0.8555 - val_loss: 0.3475 - val_accuracy: 0.8550
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3424 - accuracy: 0.8551 - val_loss: 0.3510 - val_accuracy: 0.8575


c:\Users\ps302\OneDrive\Desktop\churn_prediction\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
# Load Tensorboard
%load_ext tensorboard

In [34]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 2508), started 0:58:04 ago. (Use '!kill 2508' to kill it.)